In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',999)
import scipy.stats as sps
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# C:/User/99595/AppData/Roaming/jupyter/nbextensions/snippets/snippets.json

In [2]:
# 'skus' table
skus = pd.read_csv('Jd_com_data/JD_sku_data.csv')
# 'users' table
users = pd.read_csv('Jd_com_data/JD_user_data.csv')
# 'clicks' table
clicks = pd.read_csv('Jd_com_data/JD_click_data.csv')
# 'orders' table
orders = pd.read_csv('Jd_com_data/JD_order_data.csv')
# 'delivery' table
delivery = pd.read_csv('Jd_com_data/JD_delivery_data.csv')
# 'inventory' table
inventory = pd.read_csv('Jd_com_data/JD_inventory_data.csv')
# 'network' table
network = pd.read_csv('Jd_com_data/JD_network_data.csv')

In [4]:
users.head()

,user_ID,user_level,first_order_month,plus,gender,age,marital_status,education,city_level,purchase_power
0,000089d6a6,1,2017-08,0,F,26-35,S,3,4,3
1,0000babd1f,1,2018-03,0,U,U,U,-1,-1,-1
2,0000bc018b,3,2016-06,0,F,>=56,M,3,2,3
3,0000d0e5ab,3,2014-06,0,M,26-35,M,3,2,2
4,0000dce472,3,2012-08,1,U,U,U,-1,-1,-1


In [3]:
# 京东自营sku集合
jd1p = skus[skus['type']==1]['sku_ID'].values

orders1p = orders[orders['sku_ID'].isin(jd1p)].copy()
orders1p = orders1p[orders1p['gift_item']==0]

top100 = orders1p.groupby('sku_ID')['quantity'].sum().sort_values(ascending=False)[:100]
top100sku = top100.index.values

odrs = orders1p[orders1p['sku_ID'].isin(top100sku)] 

In [4]:
odrs['order_time'] = pd.to_datetime(odrs['order_time'])
odrs['time'] = (odrs['order_time'].dt.day*2-1)*(odrs['order_time'].dt.hour<12)+(odrs['order_time'].dt.day*2)*(odrs['order_time'].dt.hour>=12)

odrs['weekday'] = odrs['order_time'].dt.weekday+1

quantity = odrs.groupby(['sku_ID','time'],as_index=0)['quantity'].sum()

avg_original_unit_price = odrs.groupby(['sku_ID','time']).apply(
    lambda x: np.average(x['original_unit_price'], weights = x['quantity'])).reset_index(name='avg_original_unit_price')

avg_final_unit_price = odrs.groupby(['sku_ID','time']).apply(
    lambda x: np.average(x['final_unit_price'], weights = x['quantity'])).reset_index(name='avg_final_unit_price')

In [5]:
df = pd.concat([quantity,avg_original_unit_price,avg_final_unit_price],axis=1).T.drop_duplicates().T

In [6]:
df.shape

(5423, 5)

In [7]:
clicks['request_time'] = pd.to_datetime(clicks['request_time'])
clicks['time'] = (clicks['request_time'].dt.day*2-1)*(clicks['request_time'].dt.hour<12)+(clicks['request_time'].dt.day*2)*(clicks['request_time'].dt.hour>=12)

clicks['click_number'] = 1
click_number = clicks.groupby(['sku_ID','time'],as_index=0)['click_number'].sum()

clicks = click_number[click_number['sku_ID'].isin(top100sku)] 

In [8]:
clicks.shape

(6013, 3)

In [13]:
df = pd.merge(df, clicks, on = ['sku_ID','time'], how='outer')
df = tmp.sort_values(['sku_ID','time'])
df.reset_index(drop=True,inplace=True)

In [22]:
df = pd.merge(df , skus[skus['sku_ID'].isin(df['sku_ID'])][['sku_ID','type','brand_ID']] , on = ['sku_ID'])

In [25]:
df = df[[ 'time', 'sku_ID', 'type', 'brand_ID', 'click_number', 'quantity', 'avg_original_unit_price', 'avg_final_unit_price']]
df = df.rename(columns={"time": "date", "sku_ID": "product_ID", "click_number":"click_volume", "quantity":"sales"})

In [32]:
df = df[~df['click_volume'].isna()]
df = df.assign(sales=df['sales'].fillna(0))

In [40]:
def fillna_mean(x):
    x = x.assign(avg_original_unit_price=x['avg_original_unit_price'].fillna(x['avg_original_unit_price'].mean()),
                 avg_final_unit_price=x['avg_final_unit_price'].fillna(x['avg_final_unit_price'].mean()))
    return x

In [44]:
df = df.groupby('product_ID',as_index=0).apply(lambda x: fillna_mean(x))
df = df.reset_index(drop=True)

In [41]:
# def complement(tmp):
#     tmp = tmp.set_index('date')
#     missing = [i for i in range(1,63) if i not in tmp.index]
#     for m in missing:
#         tmp.loc[m]=np.NaN
#     tmp = tmp.reset_index().sort_values('date')
#     tmp = tmp.assign(product_ID=tmp['product_ID'].ffill(), type=tmp['type'].ffill(), brand_ID=tmp['brand_ID'].ffill(),           
#                click_volume=tmp['click_volume'].fillna(0), sales=tmp['sales'].fillna(0),
#                avg_original_unit_price=tmp['avg_original_unit_price'].ffill(), avg_final_unit_price = tmp['avg_final_unit_price'].ffill()
#               )
#     return tmp

# t = df.groupby('product_ID',as_index=0).apply(lambda tmp: complement(tmp)
# t = t.reset_index(drop=True)

In [47]:
df = pd.merge(df , skus[skus['sku_ID'].isin(df['product_ID'])][['sku_ID','attribute1','attribute2']] , left_on = ['product_ID'], right_on = ['sku_ID'])
df.drop('sku_ID',axis=1,inplace=True)

In [48]:
df['attribute1'] = pd.to_numeric(df['attribute1'],errors='coerce')
df['attribute2'] = pd.to_numeric(df['attribute2'],errors='coerce')
df['brand_ID'] = df['brand_ID'].astype('category').cat.codes+1

In [88]:
def cal_ma14(x):
    x1 = x.iloc[-14:].append(x)
    x1['ma14_sales'] = x1['sales'].rolling(14).mean().shift(1)
    x1 = x1.iloc[14:]
    return x1

df = df.groupby('product_ID',as_index=0).apply(lambda x: cal_ma14(x)).reset_index(drop=True)

In [106]:
weekday = odrs[['time','weekday']].drop_duplicates().reset_index(drop=True).sort_values('time')

df = pd.merge(df,weekday,left_on='date',right_on='time').sort_values(['product_ID','date']).reset_index(drop=True)
df.drop('time',axis=1,inplace=True)

In [120]:
df['attribute3'] = np.random.geometric(0.2,size=len(df))
df['attribute4'] = np.random.lognormal(mean=2, sigma=1.0, size=len(df))

In [124]:
df = df[[ 'date', 'product_ID', 'type', 'brand_ID', 'attribute1', 'attribute2','attribute3', 'attribute4',
         'click_volume', 'sales', 'avg_original_unit_price', 'avg_final_unit_price', 'ma14_sales', 'weekday']]

In [125]:
df

,date,product_ID,type,brand_ID,attribute1,attribute2,attribute3,attribute4,click_volume,sales_volume,avg_original_unit_price,avg_final_unit_price,ma14_sales_volume,weekday
0,1.0,01c9d91829,1,4,3.0,90.0,2,15.135756,504.0,14,129.000000,115.285714,12.214286,4
1,2.0,01c9d91829,1,4,3.0,90.0,7,10.096294,917.0,12,129.000000,126.916667,12.857143,4
2,3.0,01c9d91829,1,4,3.0,90.0,5,2.293909,234.0,3,129.000000,127.000000,12.785714,5
3,4.0,01c9d91829,1,4,3.0,90.0,2,11.640652,523.0,5,129.000000,129.000000,12.571429,5
4,5.0,01c9d91829,1,4,3.0,90.0,11,9.701953,216.0,0,127.689292,126.141400,10.357143,6
5,6.0,01c9d91829,1,4,3.0,90.0,6,9.198587,507.0,9,129.000000,128.000000,9.857143,6
6,7.0,01c9d91829,1,4,3.0,90.0,1,12.535721,226.0,6,129.000000,128.333333,9.071429,7
7,8.0,01c9d91829,1,4,3.0,90.0,11,1.158583,483.0,7,129.000000,129.000000,9.214286,7
8,9.0,01c9d91829,1,4,3.0,90.0,1,13.885415,228.0,5,129.000000,129.000000,8.000000,1
9,10.0,01c9d91829,1,4,3.0,90.0,1,12.405105,606.0,17,129.000000,128.411765,8.142857,1


In [127]:
df.shape

(6013, 14)

In [128]:
df.to_csv('Top100_product.csv')

========================================================================

add user information

In [11]:
tmp = pd.merge(odrs,users,on='user_ID',how='left')

In [16]:
tmp['gender'] = tmp['gender'].replace({'M':1,'F':0,'U':np.NaN})
tmp['education'] = tmp['education'].replace({-1:np.NaN})
tmp['marital_status'] = tmp['marital_status'].replace({'M':1,'S':0,'U':np.NaN})
tmp['purchase_power'] = tmp['purchase_power'].replace({-1:np.NaN})
tmp['city_level'] = tmp['city_level'].replace({-1:np.NaN})
tmp['user_level'] = tmp['user_level'].replace({-1:np.NaN})
tmp['age'] = tmp['age'].replace({'<=15':8,'16-25':20,'26-35':30,'36-45':40,'46-55':50,'>=56':62,'U':np.NaN})

In [49]:
df_user = tmp.groupby(['sku_ID','time'],as_index=0)['age','gender','education','marital_status','plus','purchase_power','user_level','city_level'].mean()

In [51]:
df = pd.read_csv('Top100_product.csv',index_col=0)

In [76]:
df = pd.merge(df,df_user,left_on=['product_ID','date'],right_on=['sku_ID','time'],how='left')
df.drop(['sku_ID','time'],axis=1,inplace=True)

In [91]:
df.shape

(6013, 22)

In [85]:
df = df[[ 'date', 'product_ID', 'type', 'brand_ID', 'attribute1', 'attribute2', 'attribute3', 'attribute4', 'click_volume',
         'avg_original_unit_price', 'avg_final_unit_price', 'ma14_sales_volume', 'weekday',
         'age', 'gender', 'education', 'marital_status', 'plus', 'purchase_power', 'user_level', 'city_level',
         'sales_volume']]

In [89]:
df.columns = ['date', 'productID', 'type', 'brandID', 'attribute1', 'attribute2',
       'attribute3', 'attribute4', 'clickVolume', 'avgOriginalUnitPrice',
       'avgFinalUnitPrice', 'ma14SalesVolume', 'weekday', 'age', 'gender',
       'education', 'maritalStatus', 'plus', 'purchasePower', 'userLevel',
       'cityLevel', 'salesVolume']

In [93]:
df = df.rename(columns={"age": "meanAge", "education": "meanEducation", "purchasePower":"meanPurchasePower", "userLevel":"meanUserLevel",
                       "cityLevel":"meanCityLevel"})

In [95]:
df = df.rename(columns={"meanPlus":"plus"})

In [97]:
df

,date,productID,type,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,weekday,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,salesVolume
0,1.0,01c9d91829,1,4,3.0,90.0,2,15.135756,504.0,129.000000,115.285714,12.214286,4,34.000000,0.230769,3.230769,0.538462,0.461538,2.230769,3.230769,2.000000,14
1,2.0,01c9d91829,1,4,3.0,90.0,7,10.096294,917.0,129.000000,126.916667,12.857143,4,30.000000,0.000000,3.000000,0.700000,0.166667,2.400000,2.416667,2.000000,12
2,3.0,01c9d91829,1,4,3.0,90.0,5,2.293909,234.0,129.000000,127.000000,12.785714,5,33.333333,0.333333,3.500000,1.000000,0.333333,2.000000,2.666667,1.000000,3
3,4.0,01c9d91829,1,4,3.0,90.0,2,11.640652,523.0,129.000000,129.000000,12.571429,5,32.500000,0.250000,2.750000,0.750000,0.200000,2.000000,2.400000,1.750000,5
4,5.0,01c9d91829,1,4,3.0,90.0,11,9.701953,216.0,127.689292,126.141400,10.357143,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,6.0,01c9d91829,1,4,3.0,90.0,6,9.198587,507.0,129.000000,128.000000,9.857143,6,32.500000,0.250000,2.857143,0.625000,0.333333,2.142857,3.777778,2.000000,9
6,7.0,01c9d91829,1,4,3.0,90.0,1,12.535721,226.0,129.000000,128.333333,9.071429,7,30.000000,0.166667,3.333333,0.500000,0.833333,2.000000,3.000000,1.833333,6
7,8.0,01c9d91829,1,4,3.0,90.0,11,1.158583,483.0,129.000000,129.000000,9.214286,7,33.333333,0.333333,2.833333,0.500000,0.142857,2.000000,2.714286,1.500000,7
8,9.0,01c9d91829,1,4,3.0,90.0,1,13.885415,228.0,129.000000,129.000000,8.000000,1,32.000000,0.400000,3.500000,1.000000,0.200000,2.000000,3.400000,2.200000,5
9,10.0,01c9d91829,1,4,3.0,90.0,1,12.405105,606.0,129.000000,128.411765,8.142857,1,30.000000,0.466667,3.000000,0.500000,0.250000,2.214286,3.000000,1.333333,17


In [98]:
df.to_csv('Top100_product.csv')